# Calculadora de Subredes


In [4]:
from IPython.display import Markdown, display, clear_output

# Instalamos las dependencias necesarias
!pip install tabulate numpy -q
!pip3 install tabulate numpy -q

import numpy as np
import tabulate as tb

clear_output()

Modifica la siguiente celda de código para especificar la dirección IP y la
máscara de red que deseas utilizar, a su vez la cantidad de subredes que deseas
obtener.


In [5]:
ip = [192, 168, 10, 0]
mask = [255, 255, 255, 0]
cantidad_de_subredes = (
    4  # Recuerda que la cantidad de subredes debe ser una potencia de 2
)

**Resultado:**


In [6]:
# Calculate the number of bits for the mask
ip.append(
    sum([f"{mask[i]:08b}".count("1") for i in range(len(mask))]),
)

# N of bits for subnets
bits_for_subnets = np.log2(cantidad_de_subredes)

if bits_for_subnets % 1:
    raise ValueError("El número de subredes debe ser una potencia de 2")

bits_for_subnets = bits_for_subnets.astype(int)

# New mask
new_mask = ip[4] + bits_for_subnets
hosts = 2 ** (32 - new_mask) - 2

# Create the new subnets
new_subnets = []
for i in range(2**bits_for_subnets):
    new_subnets.append(
        f"{i:0{bits_for_subnets}b}{'0' * (8 - bits_for_subnets)}"
    )

# Subnet mask
sub_mask = [
    255,
    255,
    255,
    sum([
        int(
            f"{2**bits_for_subnets - 1:0{bits_for_subnets}b}{'0' * (8 - bits_for_subnets)}"[
                i
            ]
        )
        * 2 ** (7 - i)
        for i in range(8)
    ]),
]

new_subnets = [
    [
        ip[0],
        ip[1],
        ip[2],
        sum([int(subnet[i]) * 2 ** (7 - i) for i in range(8)]),
    ]
    for subnet in new_subnets
]

aux_new_subnets = new_subnets.copy()
new_subnets = []

for i in range(len(aux_new_subnets)):
    first_host = aux_new_subnets[i][-1] + 1
    last_host = aux_new_subnets[i][-1] + hosts
    broadcast = aux_new_subnets[i][-1] + hosts + 1

    new_subnets.append([
        f"Subred {i + 1}",
        hosts,
        f"`{'.'.join(map(str, aux_new_subnets[i]))}`/`{new_mask}`",
        f"`{'.'.join(map(str, sub_mask))}`",
        f"`{'.'.join(map(str, aux_new_subnets[i][:3] + [first_host]))}`",
        f"`{'.'.join(map(str, aux_new_subnets[i][:3] + [last_host]))}`",
        f"`{'.'.join(map(str, aux_new_subnets[i][:3] + [broadcast]))}`",
    ])

table = tb.tabulate(
    new_subnets,
    headers=[
        "N° de subred",
        "N° de Hosts",
        "IP de la red",
        "Máscara",
        "Primer Host",
        "Último Host",
        "Broadcast",
    ],
    tablefmt="github",
)

display(Markdown(f"""
Dirección IP: `{ip[0]}.{ip[1]}.{ip[2]}.{ip[3]}/{ip[4]}`  
Máscara de red: `{mask[0]}.{mask[1]}.{mask[2]}.{mask[3]}`

Requerimientos: {cantidad_de_subredes} subredes

<div align='center'>\n\n{table}\n\n</div>"""))


Dirección IP: `192.168.10.0/24`  
Máscara de red: `255.255.255.0`

Requerimientos: 4 subredes

<div align='center'>

| N° de subred   |   N° de Hosts | IP de la red          | Máscara           | Primer Host      | Último Host      | Broadcast        |
|----------------|---------------|-----------------------|-------------------|------------------|------------------|------------------|
| Subred 1       |            62 | `192.168.10.0`/`26`   | `255.255.255.192` | `192.168.10.1`   | `192.168.10.62`  | `192.168.10.63`  |
| Subred 2       |            62 | `192.168.10.64`/`26`  | `255.255.255.192` | `192.168.10.65`  | `192.168.10.126` | `192.168.10.127` |
| Subred 3       |            62 | `192.168.10.128`/`26` | `255.255.255.192` | `192.168.10.129` | `192.168.10.190` | `192.168.10.191` |
| Subred 4       |            62 | `192.168.10.192`/`26` | `255.255.255.192` | `192.168.10.193` | `192.168.10.254` | `192.168.10.255` |

</div>